<a href="https://colab.research.google.com/github/MinsuChoKW/Data-Analytics-2025/blob/main/week6_conformance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📘 Week 6 - 적합도 평가 (Conformance Checking) (PM4Py)
- 실습 항목: Alignment 기반 적합도 평가

In [ ]:
# ✅ 환경 점검: PM4Py 버전 확인 (필요 시 설치)
try:
    import pm4py
    print("pm4py version:", pm4py.__version__)
except Exception as e:
    print("pm4py가 설치되어 있지 않거나 환경 문제가 있습니다.")
    print("Colab/로컬에서 아래 주석을 해제해 설치하세요:")
    print("!pip install -U pm4py")

## 환경 설정 (Setup)

본 노트북에서는 아래의 라이브러리를 활용한다:

* [pm4py](https://pm4py.fit.fraunhofer.de/)
* [pandas](https://pandas.pydata.org/)

In [ ]:
import pandas as pd
import pm4py

## Conformance Checking

### Process Models

프로세스 모델의 대표적인 산업 표준은 [BPMN](https://www.bpmn.org/)
이며, 이는 [BPMN.io](https://bpmn.io/) 있으며 PM4Py에 로드하여 활용할 수 있다. 본 예제에서는 `Sepsis Process` 의 기대되는 프로세스 동작을 설명하기 위해 수동으로 작성된 BPMN 모델을 불러온다. 본 프로세스는 병원에서 생명을 위협하는 패혈증이 의심되는 환자에게 수행되는 여러 물류적·진단적 활동을 묘사하는 프로세스이다. 실제 병원의 정보 시스템에서 추출된 이벤트 로그가 공개되어 있으며, 이를 실행 예제로 사용한다.

In [ ]:
sepsis_bpmn = pm4py.read_bpmn("../data/sepsis.bpmn")

In [ ]:
pm4py.vis.view_bpmn(sepsis_bpmn)

### Alignments



In [ ]:
sepsis = pd.read_csv("../data/sepsis.csv", sep=';')
num_rows = len(sepsis)
print("Number of rows: {}".format(num_rows))

In [ ]:
sepsis.head()

In [ ]:
sepsis['timestamp'] = pd.to_datetime(sepsis['timestamp'])
sepsis_log = pm4py.format_dataframe(sepsis, case_id='case_id', activity_key='activity', timestamp_key='timestamp')
sepsis_log = pm4py.convert_to_event_log(sepsis_log)

먼저, 프로세스 발견 알고리즘에서 배운 DFG 기반의 프로세스 맵을 도출한다.

In [ ]:
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.algo.discovery.dfg import algorithm as dfg_discovery
from pm4py.visualization.dfg import visualizer as dfg_visualization

dfg = dfg_discovery.apply(sepsis_log)

dfg = dfg_discovery.apply(sepsis_log, variant=dfg_discovery.Variants.PERFORMANCE)
gviz = dfg_visualization.apply(dfg, log=sepsis_log, variant=dfg_visualization.Variants.PERFORMANCE)
dfg_visualization.view(gviz)

DFG 기반의 모델은 굉장히 복잡하여 적합도 검사를 위해 직접 활용하기 쉽지 않다. 따라서, `alpha-algorithm`이나 `inductive mining`을 활용하면 적용 가능하다. 그러나, 위의 BPMN 모델로 표현된 `Sepsis Process`처럼 수작업으로 작성된 규범적인(normative) 프로세스 모델이 있는 경우에는 또 다른 새로운 인사이트를 얻을 수 있다.

PM4Py에서는 BPMN 모델을 페트리넷(Petri net)으로 변환한 후, 이벤트 로그에 포함된 트레이스(trace)들과 모델 간의 최적 정렬(optimal alignment)을 계산할 수 있다. 이를 통해, 각 트레이스마다 이벤트가 모델을 따라가는 경로(실행, run)와 어떻게 대응되는지를 확인할 수 있으며, 이는 실제 기록된 이벤트 로그와 가장 유사한 경로를 제공해준다.
(**주의**: 이 과정은 계산량이 많아 시간이 오래 걸릴 수 있다.)


In [ ]:
from pm4py.objects.conversion.bpmn import converter as bpmn_converter

sepsis_net, sepsis_im, sepsis_fm = bpmn_converter.apply(sepsis_bpmn)

sepsis_alignment = pm4py.conformance_diagnostics_alignments(sepsis_log, sepsis_net, sepsis_im, sepsis_fm)

개별적인 트레이스의 *alignment*는 아래와 같다.

In [ ]:
sepsis_alignment[0]

Alignment에 따른 통계 결과를 아래와 같이 페트리넷 위에 annotate하여 확인할 수 있다.

In [ ]:
from pm4py.visualization.petri_net import visualizer as pn_vis

pn_vis.apply(sepsis_net, sepsis_im, sepsis_fm, sepsis_log, variant = pn_vis.Variants.ALIGNMENTS)

본 통계치는 (number of skips or move on model, number of correct or synchronous events)로 확인된다.

전체 로그를 바탕으로 적용한 alignment 기반의 평균 적합도 계산 결과는 다음과 같다.

In [ ]:
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness
log_fitness = replay_fitness.evaluate(sepsis_alignment, variant=replay_fitness.Variants.ALIGNMENT_BASED)
print(log_fitness)

PM4Py를 통해 *precision*에 해당하는 결과도 연산할 수 있다.

In [ ]:
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
prec = precision_evaluator.apply(sepsis_log, sepsis_net, sepsis_im, sepsis_fm, variant=precision_evaluator.Variants.ALIGN_ETCONFORMANCE)

In [ ]:
prec